## Instance: GPStoGraph()
Takes as input the output of `GPSAnalytics().metrics.get_metrics()`

The public methods should be the following:
- `get_graphs()`
- `plot_motif()`
- `plot_graph()`
- `motif_sequence()`

In [ ]:
from xyt import GPSAnalytics, GPStoGraph

from functions_preprocessing import *

%reload_ext autoreload
%autoreload 2

## Part 1

- `GPStoGraph().get_graphs()`

In [ ]:
col_to_use = ['user_id','user_id_day','location_id','started_at','finished_at', 'lon', 'lat', 'home_location_id']
# Here we input df from part 1
nodes = pd.read_pickle('sample_data/extended_staypoint_sample_panel.pkl')[col_to_use]


act = pd.read_pickle('sample_data/staypoint_sample_panel.pkl').reset_index()
act.rename(columns={'IDNO':'user_id', 'id':'activity_id'}, inplace=True)
# Extract longitude and latitude into separate columns
act['lon'] = act['geometry'].apply(lambda point: point.x)
act['lat'] = act['geometry'].apply(lambda point: point.y)
#Parse the activity df to datetime and geopandas
act = parse_time_geo_data(act, geo_columns=['lon','lat'], datetime_format='%Y-%m-%d %H:%M:%S', CRS2='EPSG:2056')
del act['geometry']

leg = pd.read_pickle('sample_data/leg_sample_panel.pkl').reset_index()
leg.rename(columns={'id':'leg_id', 'IDNO':'user_id'}, inplace=True)
leg['started_at'] = pd.to_datetime(leg['started_at'])
leg['finished_at'] = pd.to_datetime(leg['finished_at'])

# Add the leg destination activity_id
leg = find_next_activity_id(leg, act)

# Add a 'length' column in meters
leg = gpd.GeoDataFrame(leg, geometry='geometry', crs='EPSG:4327')
leg['length'] = leg.to_crs(crs='EPSG:2056').length

# Calculate the duration in seconds and add a 'duration' column in minutes
leg['duration'] = (leg['finished_at'] - leg['started_at']).dt.total_seconds() / 60

metrics = GPSAnalytics()

staypoint1 = metrics.split_overnight(act)
staypoint2 = metrics.spatial_clustering(staypoint1)
extended_staypoint = metrics.get_metrics(staypoint2, leg)
day_staypoint = metrics.get_daily_metrics(extended_staypoint)

graphs = GPStoGraph()
multiday_graph = graphs.get_graphs(extended_staypoint)

In [ ]:
multiday_graph

## Part 2

- `GPStoGraph().plot_graph()`

In [ ]:
%%time

graphs.plot_graph(multiday_graph, path="")

## Part 3

- `GPStoGraph().plot_motif()`

In [ ]:
#parse motif_id
graphs.plot_motif(multiday_graph)

## Part 4

- `GPStoGraph().motif_sequence()`

In [ ]:
motif_seq = graphs.motif_sequence(multiday_graph)

In [ ]:
motif_seq